In [1]:
import numpy as np
import pandas as pd
train = pd.read_csv("ML/data/Titanic/train.csv")
test = pd.read_csv("ML/data/Titanic/test.csv")
full_data = [train,test]
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


In [2]:

print (train['Pclass'].value_counts(sort=False).sort_index())
print train[['Pclass','Survived']].groupby('Pclass',as_index=False).mean()

1    216
2    184
3    491
Name: Pclass, dtype: int64
   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


In [3]:

print (train['Sex'].value_counts(sort=False).sort_index())
print train[['Sex','Survived']].groupby('Sex',as_index=False).mean()

female    314
male      577
Name: Sex, dtype: int64
      Sex  Survived
0  female  0.742038
1    male  0.188908


In [4]:
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    
    age_null_count = dataset['Age'].isnull().sum()
    age_default_list = np.random.randint(low=age_avg-age_std,high=age_avg+age_std,size=age_null_count,)
    
    dataset['Age'][np.isnan(dataset['Age'])] = age_default_list
    dataset['Age'] = dataset['Age'].astype(int)



/Users/jian/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
train['CategoricalAge'] = pd.cut(train['Age'], 5)
print (train[['CategoricalAge', 'Survived']].groupby(['CategoricalAge'], as_index=False).mean())


  CategoricalAge  Survived
0  (-0.08, 16.0]  0.540541
1   (16.0, 32.0]  0.362187
2   (32.0, 48.0]  0.352490
3   (48.0, 64.0]  0.434783
4   (64.0, 80.0]  0.090909


In [6]:

print (train['SibSp'].value_counts(sort=False).sort_index())
print train[['SibSp','Survived']].groupby('SibSp',as_index=False).mean()

0    608
1    209
2     28
3     16
4     18
5      5
8      7
Name: SibSp, dtype: int64
   SibSp  Survived
0      0  0.345395
1      1  0.535885
2      2  0.464286
3      3  0.250000
4      4  0.166667
5      5  0.000000
6      8  0.000000


In [7]:

print (train['Parch'].value_counts(sort=False).sort_index())
print train[['Parch','Survived']].groupby('Parch',as_index=False).mean()

0    678
1    118
2     80
3      5
4      4
5      5
6      1
Name: Parch, dtype: int64
   Parch  Survived
0      0  0.343658
1      1  0.550847
2      2  0.500000
3      3  0.600000
4      4  0.000000
5      5  0.200000
6      6  0.000000


In [8]:
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
print (train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean())

   FamilySize  Survived
0           1  0.303538
1           2  0.552795
2           3  0.578431
3           4  0.724138
4           5  0.200000
5           6  0.136364
6           7  0.333333
7           8  0.000000
8          11  0.000000


In [9]:
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'],6)
print (train[['CategoricalFare', 'Survived']].groupby(['CategoricalFare'], as_index=False).mean())

     CategoricalFare  Survived
0    (-0.001, 7.775]  0.205128
1     (7.775, 8.662]  0.190789
2    (8.662, 14.454]  0.366906
3     (14.454, 26.0]  0.436242
4     (26.0, 52.369]  0.417808
5  (52.369, 512.329]  0.697987


In [10]:

print (train['Embarked'].value_counts(sort=False).sort_index())

C    168
Q     77
S    644
Name: Embarked, dtype: int64


In [11]:
for data in full_data:
    data['Embarked'] = data['Embarked'].fillna('S')
print (train['Embarked'].value_counts(sort=False).sort_index())

C    168
Q     77
S    646
Name: Embarked, dtype: int64


In [12]:
print (train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean())

  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.339009


In [13]:

import re
def get_title_name(name):
    title_s = re.search(' ([A-Za-z]+)\.', name)
    if title_s:
        return title_s.group(1)
    return ""
for dataset in full_data:
    dataset['TitleName'] = dataset['Name'].apply(get_title_name)
print(pd.crosstab(train['TitleName'],train['Sex']))

Sex        female  male
TitleName              
Capt            0     1
Col             0     2
Countess        1     0
Don             0     1
Dr              1     6
Jonkheer        0     1
Lady            1     0
Major           0     2
Master          0    40
Miss          182     0
Mlle            2     0
Mme             1     0
Mr              0   517
Mrs           125     0
Ms              1     0
Rev             0     6
Sir             0     1


In [14]:
train['TitleName'] = train['TitleName'].replace('Mme', 'Mrs')
train['TitleName'] = train['TitleName'].replace('Mlle', 'Miss')
train['TitleName'] = train['TitleName'].replace('Ms', 'Miss')
train['TitleName'] = train['TitleName'].replace(['Lady', 'Countess','Capt', 'Col',\
     'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
print (train[['TitleName', 'Survived']].groupby(['TitleName'], as_index=False).mean())

  TitleName  Survived
0    Master  0.575000
1      Miss  0.702703
2        Mr  0.156673
3       Mrs  0.793651
4     Other  0.347826


In [ ]:
train['IsAlone'] = 0
train.loc[train['FamilySize']==1,'IsAlone'] = 1
print (train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean())

   IsAlone  Survived
0        0  0.505650
1        1  0.303538
